In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import word_tokenize
from textblob import TextBlob
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

C:\Users\Relinns\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train = pd.read_csv('train_E6oV3lV.csv')
test = pd.read_csv('test_tweets_anuFYb8.csv')
submission = pd.read_csv('sample_submission_gfvA5FD.csv')

In [3]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [5]:
train['tweet'].shape

(31962,)

In [6]:
hypocrite = []
for i in range(len(train['tweet'])):
    if 'hypocrite' in train['tweet'][i]:
        if train['label'][i] == 1:
            hypocrite.append('racist')
        else:
            hypocrite.append('good')
    else:
        hypocrite.append('good')
df = pd.DataFrame(columns=['hypocrite'], data=hypocrite)
print(df['hypocrite'].value_counts())

train['hypocrite'] = hypocrite

good      31957
racist        5
Name: hypocrite, dtype: int64


In [7]:
train['combined'] = train['tweet'].apply(str) + ' ' + train['hypocrite'].apply(str) 

In [8]:
# review hate tweets
# for i in range(len(train['tweet'])):
#     if train['label'][i] == 1:
#         print(train['tweet'][i])

In [9]:
train.label.value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [10]:
stop_words = set(stopwords.words('english'))
lemma = WordNetLemmatizer()
stemmer = PorterStemmer()

shortcuts = {'u': 'you', 'y': 'why', 'r': 'are', 'doin': 'doing', 'hw': 'how', 'k': 'okay', 'm': 'am', 'b4': 'before',
            'idc': "i do not care", 'ty': 'thankyou', 'wlcm': 'welcome', 'bc': 'because', '<3': 'love', 'xoxo': 'love',
            'ttyl': 'talk to you later', 'gr8': 'great', 'bday': 'birthday', 'awsm': 'awesome', 'gud': 'good', 'h8': 'hate',
            'lv': 'love', 'dm': 'direct message', 'rt': 'retweet', 'wtf': 'hate', 'idgaf': 'hate',
             'irl': 'in real life', 'yolo': 'you only live once'}

def clean(text):
    text = text.lower()
    # keep alphanumeric characters only
    text = re.sub('\W+', ' ', text).strip()
    text = text.replace('user', '')
    # tokenize
    text_token = word_tokenize(text)
    # replace shortcuts using dict
    full_words = []
    for token in text_token:
        if token in shortcuts.keys():
            token = shortcuts[token]
        full_words.append(token)
    # autocorrect
#     correct_words = [str(TextBlob.correct(TextBlob(word))) for word in full_words]
    # stopwords removal
#     words = [word for word in full_words if word not in stop_words]
    # converting words to basic form 
#     stem_words = [stemmer.stem(word) for word in full_words]
    stemmed_words = [lemma.lemmatize(word) for word in full_words]
    # join list elements to string
    clean_text = " ".join(stemmed_words)
    return clean_text

In [11]:
X_train = train.combined
y_train = train.label
X_test = test.tweet

In [12]:
clean_Xtrain = [clean(sent) for sent in X_train]

In [13]:
clean_Xtrain[0]

'when a father is dysfunctional and is so selfish he drag his kid into his dysfunction run good'

In [14]:
clean_Xtest = [clean(sent) for sent in X_test]

In [15]:
print(len(clean_Xtrain))
print(len(clean_Xtest))

31962
17197


In [16]:
vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(clean_Xtrain)
X_test = vectorizer.transform(clean_Xtest)

In [17]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha=0.09)

# train model and predict
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# save it to submission csv
submission['label'] = y_pred
submission.to_csv('submission.csv', index=False)

In [18]:
import numpy as np
df = pd.DataFrame(columns=['y'])
df['y'] = y_pred
df['y'].value_counts()

0    15869
1     1328
Name: y, dtype: int64